# Parameters Section

In [1]:
# Parameters
source_workspace_name = "std-000-datamovement"
source_lakehouse_name = "lh_staging"
source_schema_name = "dbo"
source_table_name = "Person_Person"

destination_workspace_name = "std-000-datamovement"
destination_lakehouse_name = "lh_operations"
destination_schema_name = "dbo"
destination_table_name = "Person"

merge_ID_column ="BusinessEntityID"


StatementMeta(, da6dbeeb-4d2d-4c80-beea-73b8caea04c4, 3, Finished, Available, Finished)

In [2]:
# Formulate the full table names
source_full_table_name = f"`{source_lakehouse_name}`.`{source_table_name}`"
destination_full_table_name = f"`{destination_lakehouse_name}`.`{destination_table_name}`"

StatementMeta(, da6dbeeb-4d2d-4c80-beea-73b8caea04c4, 4, Finished, Available, Finished)

# Function to Merge Data Dynamically

In [3]:
def merge_data(source_full_table_name, destination_full_table_name, id_column):
    try:
        # Get existing columns from the persistent staging area table
        existing_columns = [row[0] for row in spark.sql(f"DESCRIBE {destination_full_table_name}").collect()]

        # Get columns from the source table
        source_columns = [row[0] for row in spark.sql(f"DESCRIBE {source_full_table_name}").collect()]

        # Include only the columns present in both the source and the destination tables
        columns_to_use = [col for col in source_columns if col in existing_columns]

        # Identifying the columns to be updated dynamically
        set_clause = ", ".join([f"target.{col} = source.{col}" for col in columns_to_use if col != id_column])
        insert_clause = ", ".join([f"source.{col}" for col in columns_to_use])
        insert_columns = ", ".join(columns_to_use + ['DWIsCurrent', 'DWStartDate', 'DWEndDate'])

        merge_query = f"""
        MERGE INTO {destination_full_table_name} AS target
        USING {source_full_table_name} AS source
        ON target.{id_column} = source.{id_column}
        WHEN MATCHED THEN
            UPDATE SET {set_clause}, 
            target.DWIsCurrent = true, 
            target.DWEndDate = NULL
        WHEN NOT MATCHED THEN
            INSERT ({insert_columns})
            VALUES ({insert_clause}, true, current_timestamp(), NULL)
        """
        spark.sql(merge_query)
        print(f"Data merged into {destination_full_table_name}.")

        description =f"Table {destination_full_table_name} merge statement finalized."
        status="Succeed"
        return status, description
    except Exception as e:
        print(f"An error occurred during the merge operation: {str(e)}")
        description =f"An error occurred during the merge operation: {str(e)}"
        status="Error"
        return status, description

StatementMeta(, da6dbeeb-4d2d-4c80-beea-73b8caea04c4, 5, Finished, Available, Finished)

In [9]:
def merge_data_scdtype2(source_full_table_name, destination_full_table_name, id_column):
    try:
        # Get existing columns from the destination table
        existing_columns = [row[0] for row in spark.sql(f"DESCRIBE {destination_full_table_name}").collect()]

        # Get columns from the source table
        source_columns = [row[0] for row in spark.sql(f"DESCRIBE {source_full_table_name}").collect()]

        # Include only the columns present in both the source and destination tables
        columns_to_use = [col for col in source_columns if col in existing_columns]

        # Build column comparison conditions
        change_condition = " OR ".join([f"target.{col} != source.{col}" for col in columns_to_use if col != id_column])

        # Filter source data to only include rows with changes
        source_data_with_changes = f"""
        SELECT source.*
        FROM {source_full_table_name} AS source
        LEFT JOIN {destination_full_table_name} AS target
        ON source.{id_column} = target.{id_column} AND target.DWIsCurrent = true
        WHERE {change_condition}
        """
        source_with_changes_view = "source_with_changes"
        spark.sql(f"CREATE OR REPLACE TEMP VIEW {source_with_changes_view} AS {source_data_with_changes}")

        # Construct insert columns
        insert_clause = ", ".join([f"source.{col}" for col in columns_to_use])
        insert_columns = ", ".join(columns_to_use + ['DWIsCurrent', 'DWStartDate', 'DWEndDate'])

        # Merge query without subqueries
        merge_query = f"""
        MERGE INTO {destination_full_table_name} AS target
        USING {source_with_changes_view} AS source
        ON target.{id_column} = source.{id_column} AND target.DWIsCurrent = true
        WHEN MATCHED THEN
            UPDATE SET 
                target.DWIsCurrent = false,
                target.DWEndDate = current_timestamp()
        WHEN NOT MATCHED THEN
            INSERT ({insert_columns})
            VALUES ({insert_clause}, true, current_timestamp(), NULL)
        """
        spark.sql(merge_query)

        print(f"Data merged into {destination_full_table_name}.")

        # Drop the temporary view
        spark.sql(f"DROP VIEW IF EXISTS {source_with_changes_view}")

        description = f"Table {destination_full_table_name} merge completed with change detection."
        status = "Succeed"
        return status, description
    except Exception as e:
        print(f"An error occurred during the merge operation: {str(e)}")

        # Attempt to drop the view in case of an error
        try:
            spark.sql(f"DROP VIEW IF EXISTS {source_with_changes_view}")
        except Exception as drop_error:
            print(f"Failed to drop view {source_with_changes_view}: {drop_error}")

        description = f"An error occurred during the merge operation: {str(e)}"
        status = "Error"
        return status, description


StatementMeta(, da6dbeeb-4d2d-4c80-beea-73b8caea04c4, 11, Finished, Available, Finished)

# Main Script Execution

In [10]:
from pyspark.sql import SparkSession
import json

# Merge data into the persistent staging table
status, description = merge_data_scdtype2(source_full_table_name, destination_full_table_name, merge_ID_column)

# Prepare the result as a JSON string
result = {
    "status": status,
    "description": description
}

# Check the merge status and exit with an error if the merge failed
if status == "Error":
    print(f"Error: {description}")
    raise Exception(result)

# If the merge succeeded, exit normally
mssparkutils.notebook.exit(result)

StatementMeta(, da6dbeeb-4d2d-4c80-beea-73b8caea04c4, 12, Finished, Available, Finished)

Data merged into `lh_operations`.`Person`.
ExitValue: {'status': 'Succeed', 'description': 'Table `lh_operations`.`Person` merge completed with change detection.'}